# Data Transform

In this notebook, we will ask you a series of questions to evaluate your findings from your EDA. Based on your response & justification, we will ask you to also apply a subsequent data transformation. 

If you state that you will not apply any data transformations for this step, you must **justify** as to why your dataset/machine-learning does not require the mentioned data preprocessing step.

The bonus step is completely optional, but if you provide a sufficient feature engineering step in this project we will add `1000` points to your Kahoot leaderboard score.

You will write out this transformed dataframe as a `.csv` file to your `data/` folder.

**Note**: Again, note that this dataset is quite large. If you find that some data operations take too long to complete on your machine, simply use the `sample()` method to transform a subset of your data.

In [49]:
# SMOTE Oversampling and Scaling
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# Import modules for data manipulation
from sklearn.model_selection import train_test_split

# 


import pandas as pd
import numpy as np

## Q1

Does your model contain any missing values or "non-predictive" columns? If so, which adjustments should you take to ensure that your model has good predictive capabilities? Apply your data transformations (if any) in the code-block below.

Answer here

In [44]:
# There are no missing values, but there are non-proedictive columns: nameOrig, nameDest, and isFlaggedFraud. 
# dropping these columns for simplicity and model training/performance
transacts_df = pd.read_csv('../data/bank_transactions.csv')
transacts_df.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud'], inplace=True)
transacts_df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,983.09,36730.24,35747.15,0.00,0.00,0
1,PAYMENT,55215.25,99414.00,44198.75,0.00,0.00,0
2,CASH_IN,220986.01,7773074.97,7994060.98,924031.48,703045.48,0
3,TRANSFER,2357394.75,0.00,0.00,4202580.45,6559975.19,0
4,CASH_OUT,67990.14,0.00,0.00,625317.04,693307.19,0


In [51]:
# rename coloumn for consistency
transacts_df =  transacts_df.rename(columns={'oldbalanceOrg' : 'oldbalanceOrig'})

In [52]:
# create sample of original dataset
# Sampling 10% from each class while preserving ratio
df_sample = transacts_df.groupby('isFraud', group_keys=False).apply(
    lambda x: x.sample(frac=0.2, random_state=42)
)
df_sample.head()

C:\Users\oneps\AppData\Local\Temp\ipykernel_23540\1779441535.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = transacts_df.groupby('isFraud', group_keys=False).apply(


,type,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
224997,CASH_IN,40490.46,100928.00,141418.46,242868.95,272554.26,0
851624,CASH_OUT,350188.68,5056.00,0.00,4707.66,354896.34,0
733890,PAYMENT,2224.85,2578.65,353.80,0.00,0.00,0
656179,CASH_IN,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0
347436,CASH_IN,261231.80,2215631.50,2476863.31,493557.37,232325.57,0


## Q2

Do certain transaction types consistently differ in amount or fraud likelihood? If so, how might you transform the type column to make this pattern usable by a machine learning model? Apply your data transformations (if any) in the code-block below.

According to EDA findings, the CASH_OUT and TRANSFER types are most commonly tied to fraudulent transactions
 - Below I use of one-hot enconding to transform the 'type' column into multiple binary columns for subsequent modeling

In [53]:
# One-hot enconding to transform the 'type' column into multiple binary columns for subsequent modeling
df_sample = pd.get_dummies(df_sample, columns=['type'], prefix='', prefix_sep='', drop_first=True)
df_sample.head()

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,CASH_OUT,DEBIT,PAYMENT,TRANSFER
224997,40490.46,100928.00,141418.46,242868.95,272554.26,0,False,False,False,False
851624,350188.68,5056.00,0.00,4707.66,354896.34,0,True,False,False,False
733890,2224.85,2578.65,353.80,0.00,0.00,0,False,False,True,False
656179,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0,False,False,False,False
347436,261231.80,2215631.50,2476863.31,493557.37,232325.57,0,False,False,False,False


## Q3

After exploring your data, you may have noticed that fraudulent transactions are rare compared to non-fraudulent ones. What challenges might this pose when training a machine learning model? What strategies could you use to ensure your model learns meaningful patterns from the minority class? Apply your data transformations (if any) in the code-block below.

### Challenges:

Fraudulent transactions are rare, ~0.13%, which creates a class imbalance problem. This can cause models to ignore the minority class--fraud--and achieve misleadingly high accuracy.
- Model may predict all transactions as non-fraud (99% accuracy but 0% fraud detection)
- Harder to learn minority patterns

### Possible Strategies:
- Oversample fraud cases (SMOTE)
- Apply class weights in models
- Focus on precision, recall, F1, AUC
- Feature engineering to amplify fraud signals
  - Apply Log-Tranformation?

In [54]:
X = df_sample.drop(columns=['isFraud'])
y = df_sample['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print(" Use of SMOTE to balance Fraud and no-Fraud classes.")
print(y_resampled.value_counts())


 Use of SMOTE to balance Fraud and no-Fraud classes.
isFraud
0    159793
1    159793
Name: count, dtype: int64


## Bonus (optional)

Are there interaction effects between variables (e.g., fraud and high amount and transaction type) that aren't captured directly in the dataset? Would it be helpful to manually engineer any new features that reflect these interactions? Apply your data transformations (if any) in the code-block below.

Answer Here

In [ ]:
# write out newly transformed dataset to your folder
